In [51]:

# import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import plotly.express as px
from pywt import wavedec
from pywt import waverec
from scipy.signal import hilbert
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from pywt import *
import sys



In [52]:
import sys      
sys.path.insert(0, "D:\Arai4_Projects\spinwise_project\spinewise_amanuel\ML_Project\preprocessing")

In [53]:
# import modules
from filter import apply_hilbert_filter
from filter import apply_haar_filter
from pca import apply_pca
from pca import determine_n_clusters
from segmentation import apply_segemntation
from sax import apply_symbolic_aggregation
from viz import viz_data
from viz import viz_output
from get_data import load_file
from model import kmeans
from model import db_scan
from Single_File_SAX import single_df_SAX




In [54]:
file_type = 'labeled' 
file_name = 'AW01-HBS-COMBINED-2022-11-8-14-16-48-1667918184-v1.4.3+lucina-buzz.csv'
features = ['acc_x_n','acc_y_n','acc_z_n']
preprocesses= ['filter','sax'] # write in order
model = 'k_means'
hyper_parameter = 30
number_cluster = 5


In [55]:
def start_cluster(file_type, file_name, features, preprocesses, model, clusters):
    #y = ["acc_x_n"]

    data = load_file(file_name, 'labeled')
    features_copy = features.copy()
    print(features_copy)

    features_copy.append('ts_n')
    data = data[features_copy]
    print(features)
    global cluster

    for  preprocess in preprocesses:
        if preprocess == "filter":
            
            
            data, features = apply_hilbert_filter(data,features)
            #data, features = apply_haar_filter(data, features)
        elif preprocess == "sax":
            print("##### SAX processes has activated #######")
            data, features = single_df_SAX(data, features)

        elif preprocess == "segementation":
            continue
        elif preprocess == "pca":
            print(features)
            data, features = apply_pca(data,features, 2)
        else:
            continue


    if model == "k_means":
        cluster = kmeans(data[features], number_cluster)
    elif model == "db_scan":
        cluster = db_scan(data[features], number_cluster)
    #     cluster = kmeans(data, number_cluster)
    print(features)

    data['cluster'] = cluster.labels_
    viz_output(data)
    
        




    return data



data = start_cluster(file_type, file_name, features, preprocesses, model, number_cluster)

['acc_x_n', 'acc_y_n', 'acc_z_n']
['acc_x_n', 'acc_y_n', 'acc_z_n']
##### SAX processes has activated #######
['SAX_vector']


In [56]:
data.columns

Index(['acc_x_n', 'acc_y_n', 'acc_z_n', 'ts_n', 'acc_x_n_hilbert',
       'acc_y_n_hilbert', 'acc_z_n_hilbert', 'SAX_vector', 'cluster'],
      dtype='object')

In [57]:
px.scatter(data, x='ts_n', y='acc_x_n', color='cluster')

In [60]:
d1 = load_file(file_name, 'labeled')

In [61]:
px.scatter(d1, x='ts_n', y='acc_x_n', color='label')

In [7]:
data.head()

,acc_x_n,acc_y_n,acc_z_n,ts_n,acc_x_n_hilbert,acc_y_n_hilbert,acc_z_n_hilbert,pca_0,pca_1,cluster
0,0.004883,-0.036865,-0.059082,71483.0,0.088661,0.091432,0.211696,-0.362341,-0.077630,3
1,0.004639,-0.034424,-0.062256,71484.0,0.074212,0.070665,0.179031,-0.383642,-0.107268,3
2,0.001221,-0.031738,-0.064697,71485.0,0.087537,0.068586,0.182887,-0.369772,-0.106648,3
3,0.004395,-0.029541,-0.067383,71486.0,0.077324,0.066509,0.174273,-0.381655,-0.112828,3
4,0.005615,-0.031494,-0.065674,71487.0,0.082182,0.066270,0.176942,-0.376316,-0.111345,3


In [8]:
determine_n_clusters(data, features)

[0.7763276864461988,
 0.5719105676589749,
 0.5756274914311044,
 0.4875247584865569,
 0.3519571027753299,
 0.3526217536732691,
 0.3892179688580129,
 0.39691240592132815,
 0.4033843621365232]

In [9]:
d_sax= single_df_SAX(data,features)

In [10]:
d_sax.head()

,acc_x_n,acc_y_n,acc_z_n,ts_n,acc_x_n_hilbert,acc_y_n_hilbert,acc_z_n_hilbert,pca_0,pca_1,cluster,SAX_vector
0,0.004883,-0.036865,-0.059082,71483.0,0.088661,0.091432,0.211696,-0.362341,-0.077630,3,0.157311
1,0.004639,-0.034424,-0.062256,71484.0,0.074212,0.070665,0.179031,-0.383642,-0.107268,3,0.157311
2,0.001221,-0.031738,-0.064697,71485.0,0.087537,0.068586,0.182887,-0.369772,-0.106648,3,0.157311
3,0.004395,-0.029541,-0.067383,71486.0,0.077324,0.066509,0.174273,-0.381655,-0.112828,3,0.157311
4,0.005615,-0.031494,-0.065674,71487.0,0.082182,0.066270,0.176942,-0.376316,-0.111345,3,0.157311


In [18]:
px.scatter(d_sax, x='ts_n', y='acc_z_n',color = 'SAX_vector')

In [15]:
d_sax['SAX_vector'].value_counts()

 0.157311    4210
 0.488776    1900
-0.157311    1423
-1.534121     712
-0.488776     298
-0.887147     269
 0.887147      89
 0.000000      24
Name: SAX_vector, dtype: int64

In [16]:
import gradio as gr
import os

def start_cluster(start_date, end_date, amount, google_percentage, tsla_percentage, msft_percentage):
    pass


with gr.Blocks() as demo:
    
    txt_0= gr.Checkbox(label="file type")
    txt_1= gr.Dropdown(label="file name")
    txt_2= gr.Textbox(label="Amount")
    txt_3= gr.Textbox(label="Google %")
    txt_4= gr.Textbox(label="Tesla %")
    txt_5= gr.Textbox(label="microsoft%")
    txt_6= gr.TextArea(value="", label="Output")
    btn = gr.Button(value="Submit")
    btn.click(start_cluster, inputs=[txt_0,txt_1,txt_2,txt_3,txt_4,txt_5], outputs=[txt_6])
    

if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [5]:
def start_cluster(file_name, features, proceses, model):
    # open the file
    # prepare the file
    # call the proceses
    # apply test the model

    

start_cluster(data,features, k_means)

NameError: name 'preprocess_data' is not defined

In [ ]:
def train_kmeans():
    """train a model
    
    """
    #data= applying_pca(df)
    model = KMeans(n_clusters=5)
    data = df[['gyr_x_n_hilbert','gyr_y_n_hilbert','gyr_z_n_hilbert']]
    model.fit(data)
    return model

In [ ]:
def train_db_scan():
    pass